In [28]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

In [29]:
# Load the saved Keras model
model = load_model('models/dataset1model.h5')

In [30]:
# Define the image folder path and output CSV file path
image_folder = 'dataset1/unknown_images'
#image_folder = 'dataset2/images'
csv_path = 'dataset1/unknown_images/results.csv'
#csv_path = 'dataset2/images/results.csv'

In [31]:
# Initialize empty lists to store the image names and predicted classes
image_names = []
predicted_classes = []
accuracies = []

In [37]:

# Iterate through all images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load the image and preprocess it
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (256, 256))
        image = np.expand_dims(image, axis=0)
        image = image / 255.0

        # Make predictions on the image using the loaded model
        predictions = model.predict(image)
        
       

        # Get the index of the class with the highest probability
        class_index = np.argmax(predictions)

        # Get the accuracy of the prediction
        accuracy = predictions[0][class_index]
        
         # Get the predicted class and probability
        if predictions < 0.5: 
            mclass = 'Normal'
        else:
            mclass = 'Sick'

            
        # Calculate accuracy, recall, and F1 score
#         true_label = ground_truth_df[ground_truth_df['Image'] == filename]['Label'].values[0]
#         accuracy = accuracy_score([true_label], [mclass])
#         recall = recall_score([true_label], [mclass], pos_label='Sick')
#         f1 = f1_score([true_label], [mclass], pos_label='Sick')
        
        # Add the image name and predicted class to the lists
        image_names.append(filename)
        predicted_classes.append(mclass)
#         accuracies.append(accuracy)
#         recalls.append(recall)
#         f1_scores.append(f1)
        
        # Print the filename and predicted class
        print(f'{filename}, {mclass}')
                

1.png, Normal
10.png, Sick
11.png, Normal
12.png, Sick
13.png, Normal
14.png, Sick
15.png, Normal
16.png, Sick
17.png, Normal
18.png, Sick
19.png, Normal
2.png, Sick
20.png, Sick
21.png, Normal
22.png, Sick
23.png, Normal
24.png, Sick
25.png, Normal
26.png, Sick
27.png, Sick
28.png, Sick
29.png, Normal
3.png, Normal
30.png, Sick
31.png, Normal
32.png, Sick
33.png, Normal
34.png, Sick
35.png, Normal
36.png, Sick
37.png, Normal
38.png, Sick
39.png, Normal
4.png, Sick
40.png, Sick
41.png, Normal
42.png, Sick
43.png, Normal
44.png, Sick
45.png, Normal
46.png, Sick
47.png, Normal
48.png, Sick
49.png, Normal
5.png, Normal
50.png, Sick
6.png, Sick
7.png, Normal
8.png, Sick
9.png, Normal


In [20]:
# Create a results DataFrame
# results_df = pd.DataFrame({
#     'Image': image_names,
#     'Predicted Class': predicted_classes,
#     'Accuracy': accuracies,
#     'Recall': recalls,
#     'F1 Score': f1_scores
# })

# Save the results to a CSV file
# results_df.to_csv(csv_path, index=False)

## Test Model on Dataset 2

In [22]:
import tensorflow as tf

In [23]:
from tensorflow.keras.models import load_model
model = load_model('models/dataset1model.h5')

In [24]:
test_data = tf.keras.preprocessing.image_dataset_from_directory('dataset2/images')
test_data = test_data.map(lambda x,y: (x/255, y))
test_data.as_numpy_iterator().next()

Found 224 files belonging to 2 classes.


(array([[[[0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         [[0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         [[0.02549235, 0.02549235, 0.02549235],
          [0.02148641, 0.02148641, 0.02148641],
          [0.01671346, 0.01671346, 0.01671346],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         ...,
 
         [[0.        , 0.        , 0.        ],
          [0.     

In [26]:
results = model.evaluate(test_data)

7/7 [==============================] - 1s 112ms/step - loss: 7.9132 - accuracy: 0.4464


In [27]:
print('Test Loss:', results[0])
print('Test Accuracy:', results[1])

Test Loss: 7.913227081298828
Test Accuracy: 0.4464285671710968


In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
test_data_iterator = test_data.as_numpy_iterator()
y_true = np.concatenate([y for _, y in test_data_iterator])
y_pred = np.round(model.predict(test_data)).astype(int).flatten()
print('Precision:', precision_score(y_true, y_pred))
print('Recall:', recall_score(y_true, y_pred))
print('F1-Score:', f1_score(y_true, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_true, y_pred))

Precision: 0.49528301886792453
Recall: 0.9375
F1-Score: 0.6481481481481481
Confusion Matrix:
 [[  5 107]
 [  7 105]]


In [14]:
# Define file paths for test data
test_file_paths = [f[0] for f in test_data]

# Generate predictions
y_pred_prob = model.predict(test_data)
y_pred = np.round(y_pred_prob).astype(int).flatten()

# Create results DataFrame and merge with file paths
results_df = pd.DataFrame({'Actual Class': y_true, 'Predicted Class': y_pred, 'Accuracy': y_pred_prob.flatten()})
results_df['Filename'] = test_file_paths
results_df = results_df[['Filename', 'Actual Class', 'Predicted Class', 'Accuracy']]

# Write results to CSV
results_df.to_csv('dataset2_results.csv', index=False)

ValueError: Length of values does not match length of index